In [ ]:
import os
from glob import glob
import numpy as np
import gc

In [ ]:
!pip install -q torchsummaryX wandb --quiet

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import sklearn.metrics
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
from torch import optim
from torch.nn import init
import wandb
from torchsummaryX import summary
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device:",device)


#from torchsummaryX import summary

from tqdm.notebook import tqdm

Device: cuda


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install --upgrade --force-reinstall --no-deps kaggle==1.5.8
!mkdir /root/.kaggle

with open("/root/.kaggle/kaggle.json", "w+") as f:
    f.write('{"username":"adritadas","key":"4d8e5e73b5126973227f2e938ba1249f"}') 
    # Put your kaggle username & key here

!chmod 600 /root/.kaggle/kaggle.json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached kaggle-1.5.8-py3-none-any.whl
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.8
    Uninstalling kaggle-1.5.8:
      Successfully uninstalled kaggle-1.5.8
mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
!kaggle competitions download -c 11-785-s23-hw1p2
!mkdir '/content/data'

!unzip -qo '11-785-s23-hw1p2.zip' -d '/content/data'

11-785-s23-hw1p2.zip: Skipping, found more recently modified local copy (use --force to force download)
mkdir: cannot create directory ‘/content/data’: File exists


In [ ]:
data_dir_dev_clean = "/content/data/11-785-s23-hw1p2/dev-clean"

data_dir_train_clean = "/content/data/11-785-s23-hw1p2/train-clean-360"

In [ ]:
root = "/content/data/11-785-s23-hw1p2/"
partition = "dev-clean"

In [ ]:
PHONEMES = [
            '[SIL]',   'AA',    'AE',    'AH',    'AO',    'AW',    'AY',  
            'B',     'CH',    'D',     'DH',    'EH',    'ER',    'EY',
            'F',     'G',     'HH',    'IH',    'IY',    'JH',    'K',
            'L',     'M',     'N',     'NG',    'OW',    'OY',    'P',
            'R',     'S',     'SH',    'T',     'TH',    'UH',    'UW',
            'V',     'W',     'Y',     'Z',     'ZH']

print(PHONEMES.index('[SIL]'))  

0


In [ ]:
torch.cuda.empty_cache()
gc.collect()

0

In [ ]:
mfcc_dir = sorted(glob(os.path.join(root, partition, 'mfcc','*.npy')))
transcript_dir = sorted(glob(os.path.join(root,partition,'transcript','*.npy')))

In [ ]:
class AudioDataset(torch.utils.data.Dataset):
  def __init__(self, root, phonemes = PHONEMES, context=0, partition= "train-clean-360"): 
    self.context = context
    self.phonemes = phonemes

    mfcc_names =  sorted(glob(os.path.join(root, partition, 'mfcc','*.npy')))
    # TODO: List files in self.transcript_dir using os.listdir in sorted order
    transcript_names  = sorted(glob(os.path.join(root,partition,'transcript','*.npy')))

        

    # Making sure that we have the same no. of mfcc and transcripts
    assert len(mfcc_names) == len(transcript_names)
    length = len(mfcc_names)
    T = 0 
    for i in range(length):
      mfcc = np.load(mfcc_names[i])
      T += mfcc.shape[0]

    self.mfccs = np.zeros((T + 2*context, 27),dtype=np.float32)
    self.transcripts = np.zeros((T, ),dtype=np.uint8)
    self.length = T
    cx, cy = context, 0
    for i in range(length):
      mfcc = np.load(mfcc_names[i])
      mfcc -= (np.mean(mfcc, axis=0) + 1e-8)
      transcript = np.load(transcript_names[i])[1:-1]
      transcript = [PHONEMES.index(transcript[i])for i in range (len(transcript))]
      T_i = mfcc.shape[0]
      self.mfccs[cx:cx+T_i] = mfcc
      self.transcripts[cy:cy+T_i] = transcript
      cx += T_i
      cy += T_i
   
  def __len__(self):
    return self.length

  def __getitem__(self, ind):
        
        # TODO: Based on context and offset, return a frame at given index with context frames to the left, and right.
    mfccs =  self.mfccs[ind:ind+2*self.context+1, :].reshape(-1)
        # After slicing, you get an array of shape 2*context+1 x 27. But our MLP needs 1d data and not 2d.
        
    mfccs      = torch.FloatTensor(mfccs) # Convert to tensors
    transcripts    = torch.tensor(self.transcripts[ind])       

    return mfccs, transcripts

In [ ]:
class AudioTestDataset(torch.utils.data.Dataset):
  def __init__(self, root, phonemes = PHONEMES, context=25, partition= "train-clean-360"): 
    self.context = context
    self.phonemes = phonemes

    mfcc_names =  sorted(glob(os.path.join(root, partition, 'mfcc','*.npy')))
    # TODO: List files in self.transcript_dir using os.listdir in sorted order
    

        

    # Making sure that we have the same no. of mfcc and transcripts
    
    length = len(mfcc_names)
    T = 0 
    for i in range(length):
      mfcc = np.load(mfcc_names[i])
      mfcc -= (np.mean(mfcc, axis=0) + 1e-8)
      T += mfcc.shape[0]

    self.mfccs = np.zeros((T + 2*context, 27),dtype=np.float32)
    self.length = T
    cx = context
    for i in range(length):
      mfcc = np.load(mfcc_names[i])
      T_i = mfcc.shape[0]
      self.mfccs[cx:cx+T_i] = mfcc
      cx += T_i

  def __len__(self):
    return self.length

  def __getitem__(self, ind):
    mfccs =  self.mfccs[ind:ind+2*self.context+1, :].reshape(-1)
        # After slicing, you get an array of shape 2*context+1 x 27. But our MLP needs 1d data and not 2d.
        

    mfccs      = torch.FloatTensor(mfccs) # Convert to tensors
        
    return mfccs

In [ ]:
config = {
    'epochs'        : 60,
    'batch_size'    : 1024*8,
    'context'       : 25,
    'init_lr'       : 1e-3,
    'architecture'  : 'unkown'
    # Add more as you need them - e.g dropout values, weight decay, scheduler parameters
}

In [ ]:
torch.cuda.empty_cache()
gc.collect()

0

In [ ]:
#TODO: Create a dataset object using the AudioDataset class for the training data 
train_data = AudioDataset(root, phonemes = PHONEMES, context=config['context'], partition= "train-clean-360")

# TODO: Create a dataset object using the AudioDataset class for the validation data 
val_data = AudioDataset(root, phonemes = PHONEMES, context=config['context'], partition= "dev-clean")

# TODO: Create a dataset object using the AudioTestDataset class for the test data 
test_data = AudioTestDataset(root, phonemes = PHONEMES, context=config['context'], partition= "test-clean")

In [ ]:
train_loader = torch.utils.data.DataLoader(
    dataset     = train_data, 
    num_workers = 4,
    batch_size  = config['batch_size'], 
    pin_memory  = True,
    shuffle     = True
)

val_loader = torch.utils.data.DataLoader(
    dataset     = val_data, 
    num_workers = 2,
    batch_size  = config['batch_size'],
    pin_memory  = True,
    shuffle     = False
)

test_loader = torch.utils.data.DataLoader(
    dataset     = test_data, 
    num_workers = 2, 
    batch_size  = config['batch_size'], 
    pin_memory  = True, 
    shuffle     = False
)


print("Batch size     : ", config['batch_size'])
print("Context        : ", config['context'])
print("Input size     : ", (2*config['context']+1)*27)
print("Output symbols : ", len(PHONEMES))

print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))
print("Validation dataset samples = {}, batches = {}".format(val_data.__len__(), len(val_loader)))
print("Test dataset samples = {}, batches = {}".format(test_data.__len__(), len(test_loader)))

Batch size     :  8192
Context        :  25
Input size     :  1377
Output symbols :  40
Train dataset samples = 130453995, batches = 15925
Validation dataset samples = 1928204, batches = 236
Test dataset samples = 1934138, batches = 237


In [ ]:
# Implement dropout,scheduler,kaiming
class TrainModel(torch.nn.Module):
     def __init__(self,in_features,out_features):
        super(TrainModel,self).__init__()

        self.model = torch.nn.Sequential(
            
            torch.nn.Linear(in_features , 2100),
            torch.nn.BatchNorm1d(2100),
            torch.nn.Softplus(),
            torch.nn.Dropout(p=0.15),
            
            torch.nn.Linear(2100,2100),
            torch.nn.BatchNorm1d(2100),
            torch.nn.Softplus(),
            torch.nn.Dropout(p=0.15),
            
            torch.nn.Linear(2100,2100),
            torch.nn.BatchNorm1d(2100),
            torch.nn.Softplus(),
            torch.nn.Dropout(p=0.15),
            
            torch.nn.Linear(2100,2100),
            torch.nn.BatchNorm1d(2100),
            torch.nn.Softplus(),
            torch.nn.Dropout(p=0.2),
            
            torch.nn.Linear(2100,2100),
            torch.nn.BatchNorm1d(2100),
            torch.nn.Softplus(),
            torch.nn.Dropout(p=0.2),
            
            torch.nn.Linear(2100,out_features),
          
            

            
        )

     def forward(self,x):
       out = self.model(x)

       return out

    

In [ ]:
for i, data in enumerate(train_loader):
    mfccs, transcripts = data
    print(mfccs.shape, transcripts.shape)
    break

torch.Size([8192, 1377]) torch.Size([8192])


In [ ]:
in_features = (2*config['context'] + 1 ) * 27
out_features = len(train_data.phonemes)
model = TrainModel(in_features,out_features).to(device)
print(model)
summary(model,mfccs.to(device))

TrainModel(
  (model): Sequential(
    (0): Linear(in_features=1377, out_features=2100, bias=True)
    (1): BatchNorm1d(2100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Softplus(beta=1, threshold=20)
    (3): Dropout(p=0.15, inplace=False)
    (4): Linear(in_features=2100, out_features=2100, bias=True)
    (5): BatchNorm1d(2100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): Softplus(beta=1, threshold=20)
    (7): Dropout(p=0.15, inplace=False)
    (8): Linear(in_features=2100, out_features=2100, bias=True)
    (9): BatchNorm1d(2100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): Softplus(beta=1, threshold=20)
    (11): Dropout(p=0.15, inplace=False)
    (12): Linear(in_features=2100, out_features=2100, bias=True)
    (13): BatchNorm1d(2100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (14): Softplus(beta=1, threshold=20)
    (15): Dropout(p=0.2, inplace=False)
    (16): Linear(in_

/usr/local/lib/python3.8/dist-packages/torchsummaryX/torchsummaryX.py:101: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_sum = df.sum()


,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_model.Linear_0,"[1377, 2100]","[8192, 2100]",2893800.0,2891700.0
1_model.BatchNorm1d_1,[2100],"[8192, 2100]",4200.0,2100.0
2_model.Softplus_2,-,"[8192, 2100]",NaN,NaN
3_model.Dropout_3,-,"[8192, 2100]",NaN,NaN
4_model.Linear_4,"[2100, 2100]","[8192, 2100]",4412100.0,4410000.0
5_model.BatchNorm1d_5,[2100],"[8192, 2100]",4200.0,2100.0
6_model.Softplus_6,-,"[8192, 2100]",NaN,NaN
7_model.Dropout_7,-,"[8192, 2100]",NaN,NaN
8_model.Linear_8,"[2100, 2100]","[8192, 2100]",4412100.0,4410000.0


In [ ]:
criterion = torch.nn.CrossEntropyLoss(label_smoothing=0.1)

optimizer = torch.optim.AdamW(model.parameters(), lr = config['init_lr'])
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer,
                                             step_size=6, gamma=0.5)
scaler = torch.cuda.amp.GradScaler()

In [ ]:
torch.cuda.empty_cache()
gc.collect()

911

In [ ]:
def train(model, dataloader, optimizer,criterion,scaler):

  model.train()
  tloss,tacc = 0,0
  batch_bar = tqdm(total=len(train_loader),dynamic_ncols=True,leave=False,position=0,desc='Train')

  for i, (mfccs,transcripts) in enumerate(dataloader):

    optimizer.zero_grad()

    mfccs = mfccs.to(device)
    transcripts = transcripts.to(device)

    with torch.autocast(device_type='cuda', dtype=torch.float16):
      logits = model(mfccs)
      loss = criterion(logits,transcripts)
    

    #loss.backward()
    scaler.scale(loss).backward()

    #optimizer.step()
    scaler.step(optimizer)
    scaler.update()

    tloss += loss.item()
    tacc  += torch.sum(torch.argmax(logits, dim=1)==transcripts).item()/logits.shape[0]

    batch_bar.set_postfix(loss = "{:.04f}".format(float(tloss/(i+1))),
                           acc ="{:.04f}%".format(float(tacc*100/(i+1))))
    batch_bar.update()

    del mfccs,transcripts,logits
    torch.cuda.empty_cache()

  batch_bar.close()
  tloss /= len(train_loader)
  tacc /= len(train_loader)

  return tloss,tacc


In [ ]:
def eval(model, dataloader, optimizer,criterion):

  model.eval()
  vloss,vacc = 0,0
  batch_bar = tqdm(total=len(val_loader),dynamic_ncols=True,leave=False,position=0,desc='Train')

  for i, (mfccs,transcripts) in enumerate(dataloader):

   

    mfccs = mfccs.to(device)
    transcripts = transcripts.to(device)
    with torch.inference_mode():
      logits = model(mfccs)
      loss = criterion(logits,transcripts)

   

  
    vloss += loss.item()
    vacc  += torch.sum(torch.argmax(logits, dim=1)==transcripts).item()/logits.shape[0]

    batch_bar.set_postfix(loss = "{:.04f}".format(float(vloss/(i+1))),
                           acc ="{:.04f}%".format(float(vacc*100/(i+1))))
    batch_bar.update()

    del mfccs,transcripts,logits
    torch.cuda.empty_cache()

  batch_bar.close()
  vloss /= len(val_loader)
  vacc /= len(val_loader)

  return vloss,vacc

In [ ]:
wandb.login(key="e2fbcbd28e7f2e0bdeb1ab36f90d65664c335bd7")

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: adritad (iw). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
run = wandb.init(
    name    = "360-run2", ### Wandb creates random run names if you skip this field, we recommend you give useful names
    reinit  = True, ### Allows reinitalizing runs when you re-run this cell
    #id     = "y28t31uz", ### Insert specific run id here if you want to resume a previous run
    #resume = "must", ### You need this to resume previous runs, but comment out reinit = True when using this
    project = "hw1p2", ### Project should be created in your wandb account 
    config  = config ### Wandb Config for your run
)

In [ ]:
model_arch  = str(model)

### Save it in a txt file 
arch_file   = open("model_arch.txt", "w")
file_write  = arch_file.write(model_arch)
arch_file.close()

### log it in your wandb run with wandb.save()
wandb.save('model_arch.txt')

['/content/wandb/run-20230227_204932-zaruvj2t/files/model_arch.txt']

In [ ]:
torch.cuda.empty_cache()
gc.collect()

optimizer.param_groups[0]['lr'] = 1e-3

config['epochs'] = 60
for epoch in range(config['epochs']):

  print("\nEpoch {}/{}".format(epoch+1,config['epochs']))

  curr_lr = float(optimizer.param_groups[0]['lr'])
  train_loss, train_acc = train(model,train_loader,optimizer,criterion,scaler)
  val_loss,val_acc = eval(model,val_loader,optimizer,criterion)

  print("\tTrain Acc {:.04f}%\tTrain Loss {:.04f}\t Learning Rate {:.07f}".format(train_acc*100, train_loss,curr_lr))
  print("\tVal Acc {:.04f}%\tVal Loss {:.04f}".format(val_acc*100 , val_loss))
  torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'Accuracy': val_acc,
            },"/content/drive/MyDrive/checkpoint/chk_"+str(epoch)+".pth")


In [ ]:
#checkpoint = torch.load("/content/drive/MyDrive/checkpoint/chk_"+str(40)+".pth")
#model.load_state_dict(checkpoint['model_state_dict'])
#optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
#optimizer.param_groups[0]['lr'] = 1e-5
#e = checkpoint['epoch'] + 1

In [ ]:
def test(model, dataloader,optimizer,criterion):
    ### What you call for model to perform inference?
    model.eval()

    ### List to store predicted phonemes of test data
    test_predictions = []

    ### Which mode do you need to avoid gradients?
    with torch.inference_mode():

        for i, mfccs in enumerate(tqdm(test_loader)):

            mfccs   = mfccs.to(device)             
            
            logits  = model(mfccs)

            ### Get most likely predicted phoneme with argmax
            predicted_labels = torch.argmax(model(mfccs), dim=1)
            test_predictions.extend(list(predicted_labels.detach().cpu().numpy()))

            ### How do you store predicted_phonemes with test_predictions? Hint, look at eval 
            # TODO

    return test_predictions

In [ ]:
test_predictions = test(model,test_loader,optimizer,criterion) 

In [ ]:
### Create CSV file with predictions
with open("./submission.csv", "w+") as f:
    f.write("id,label\n")
    for i in range(len(test_predictions)):
        f.write("{},{}\n".format(i, PHONEMES[test_predictions[i]]))

In [ ]:
!kaggle competitions submit -c 11-785-s23-hw1p2 -f ./submission.csv -m "Test Submission"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
